# **DataFrame Films**

## **Importations datasets**

In [1]:
import pandas as pd
#! === IMPORT ===


#* Les datasets importants
df_title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
df_title_ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')

#* Les datasets intéressants
df_title_akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
df_title_crew = pd.read_csv('https://datasets.imdbws.com/title.crew.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
df_name_basics = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
df_title_principals = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', sep='\t', compression='gzip', na_values='\\N')

C:\Users\koke7\AppData\Local\Temp\ipykernel_31708\2730036939.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
C:\Users\koke7\AppData\Local\Temp\ipykernel_31708\2730036939.py:10: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')


## **Initialisation Principal**

In [6]:

#! === SETUP ===
#? title_basics

#* Garder uniquement les films
df_title_basics_clean = df_title_basics[df_title_basics['titleType'] == 'movie']

#* Supprimer les lignes avec des données manquantes essentielles
df_title_basics_clean = df_title_basics_clean.dropna(subset=['startYear', 'genres'])

#* Convertir startYear en entier et filtrer les années valides
df_title_basics_clean['startYear'] = df_title_basics_clean['startYear'].astype(int)
df_title_basics_clean = df_title_basics_clean[(df_title_basics_clean['startYear'] >= 1970) & (df_title_basics_clean['startYear'] <= 2025)]

#* Garder les colonnes essentiels
df_title_basics_clean = df_title_basics_clean[['tconst', 'primaryTitle', 'startYear', 'genres', 'runtimeMinutes']]



#! === SETUP ===
#? title_rating

#* Supprimer les films avec peu de votes
df_title_ratings_clean = df_title_ratings[df_title_ratings['numVotes'] > 1000]



#! === SETUP ===
#? title_akas

#* Garder uniquement les titres de film français
df_title_akas_clean = df_title_akas[df_title_akas['region'] == 'FR']

#* Garder les colonnes essentiels
df_title_akas_clean = df_title_akas_clean[['titleId', 'title']]

#* Renommer l'identifiant
df_title_akas_clean.rename(columns={'titleId' : 'tconst'}, inplace=True)



#! === SETUP ===
#? title_crew

#* Remplir les valeurs manquantes avec un indicateur, par exemple 'Unknown'
df_title_crew_clean = df_title_crew.fillna({'directors': 'Unknown'})

#* Transformer la colonne 'directors' en une liste
df_title_crew_clean['directors'] = df_title_crew_clean['directors'].str.split(',')

#* Exploser la colonne 'directors' en lignes individuelles
df_title_crew_clean = df_title_crew_clean.explode('directors')

#* Fusionner les 'directors' avec 'name_basics' pour obtenir les 'primaryName'
title_crew_with_directors = df_title_crew_clean.merge(df_name_basics[['nconst', 'primaryName']], 
                                                     how='left', 
                                                     left_on='directors', 
                                                     right_on='nconst')

#* Regrouper les résultats par 'tconst' pour remettre les réalisateurs sous forme de liste
title_crew_with_directors = title_crew_with_directors.groupby('tconst')['primaryName'].apply(list).reset_index()



#! === SETUP ===
#? title_principals

#* Supprimer les lignes avec des valeurs manquantes dans 'category'
df_title_principals_clean = df_title_principals.dropna(subset=['category'])

#* Filtrer les acteurs dans title.principals
df_title_principals_clean = df_title_principals_clean[df_title_principals_clean['category'] == 'actor']

#* Fusionner avec name.basics pour obtenir les noms des acteurs
actors_with_names = pd.merge(df_title_principals_clean, df_name_basics[['nconst', 'primaryName']], on='nconst', how='left')

#* Créer une liste d'acteurs par film
actors_list_per_film = actors_with_names.groupby('tconst')['primaryName'].apply(list).reset_index()

## **Fusion Principal**

In [9]:

#! === MERGEUP ===


#* Merge de title_basics & title_ratings
df_merged_v1 = df_title_basics_clean.merge(df_title_ratings_clean, on="tconst")

#* Merge de df_merged_v1 & df_title_akas
df_merged_v2 = df_merged_v1.merge(df_title_akas_clean, on='tconst', how='left')

#* Merge de df_merged_v2 & title_crew_with_directors
df_merged_v3 = df_merged_v2.merge(title_crew_with_directors, on='tconst', how='left')

#* Merge de df_merged_v3 & actors_list_per_film
df_merged_v3['actors'] = df_merged_v3['tconst'].map(dict(zip(actors_list_per_film['tconst'], actors_list_per_film['primaryName'])))

## **Exportation**

In [10]:

#! === EXPORT ===


df_merged_v3.to_csv("../data/raw/df_movie.csv", index=False)

## **Importation**

In [16]:
import pandas as pd

#! === IMPORT ===


file_path = "../data/raw/df_movie.csv"

df_movie = pd.read_csv(file_path)

## **Mise en page**

In [17]:

#! === RENAMEUP ===


#* Renommer les colonnes de title_basics
df_movie.rename(columns={'tconst': 'ID',
                         'primaryTitle': 'Titre Original',
                         'startYear': 'Année',
                         'genres':  'Genres', 
                         'runtimeMinutes': 'Durée (minutes)',
                         'averageRating': 'Note',
                         'numVotes': 'Votes',
                         'title': 'Titre Français',
                         'actors': 'Acteurs',
                         'primaryName': 'Réalisateur(s)'}, inplace=True)

df_movie['Réalisateur(s)'] = df_movie['Réalisateur(s)'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_movie['Réalisateur(s)'] = df_movie['Réalisateur(s)'].str.strip("[]").str.replace("'", "")

df_movie['Acteurs'] = df_movie['Acteurs'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_movie['Acteurs'] = df_movie['Acteurs'].str.strip("[]").str.replace("'", "")

df_movie['Genres'] = df_movie['Genres'].str.replace(",", ", ")

## **Ajout de données supplémentaires**

In [19]:

#! === ADDUP ===


#* Ajout de la colonne 'popularité'
def categorize_votes(votes):
    if votes < 50000:
        return "Connu"
    elif 50000 <= votes < 200000:
        return "Populaire"
    elif 200000 <= votes < 1000000:
        return "Très populaire"
    else:
        return "Blockbuster"

df_movie['Popularité'] = df_movie['Votes'].apply(categorize_votes)

#* Ajout de la colonne "métrage"
def categorize_times(duree):
    if duree < 100:
        return "Court"
    elif 100 <= duree <= 200:
        return "Moyen"
    else:
        return "Long"

df_movie['Métrage'] = df_movie['Durée (minutes)'].apply(categorize_times)

#* Ajout de la colonne décennie
def categorize_years(year):
    if 1970 <= year <= 1979:
        return "70°s"
    if 1980 <= year <= 1989:
        return "80°s"
    if 1990 <= year <= 1999:
        return "90°s"
    if 2000 <= year <= 2009:
        return "2000"
    if 2010 <= year <= 2019:
        return "2010"
    if 2020 <= year <= 2029:
        return "2020"

df_movie['Décennie'] = df_movie['Année'].apply(categorize_years)

#* Ajout d'une genre principal
df_movie['Genre Principal'] = df_movie['Genres'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else x)

#* Repositionner les colonnes
df_movie = df_movie[['ID', 'Réalisateur(s)', 'Titre Original', 'Titre Français', 'Métrage', 'Durée (minutes)', 'Année', 'Décennie', 'Genres', 'Genre Principal', 'Note', 'Votes', 'Popularité', 'Acteurs']]

## **Processus de nettoyage**

In [20]:

#! === CLEANUP V1 ===


#* Supprimer les doublons
df_movie_cleaned_v1 = df_movie.drop_duplicates(subset='ID')

#* Convertir en numéric
df_movie_cleaned_v1["Durée (minutes)"] = pd.to_numeric(df_movie_cleaned_v1["Durée (minutes)"], errors="coerce") # Converti en numérique

#* Supprimer les films avec plus de 300 minutes
df_movie_cleaned_v1 = df_movie_cleaned_v1[df_movie_cleaned_v1['Durée (minutes)'] < 300]

#* Supprimer les films avec moins de 60 minutes
df_movie_cleaned_v1 = df_movie_cleaned_v1[df_movie_cleaned_v1['Durée (minutes)'] > 60]

#* Supprimer les films avec moins de 15 000 votes
df_movie_cleaned_v1 = df_movie_cleaned_v1[df_movie_cleaned_v1['Votes'] > 4000]

#* Remplacer les valeurs manquantes dans 'Titre Français' par les valeurs de 'Titre'
df_movie_cleaned_v1['Titre Français'] = df_movie_cleaned_v1['Titre Français'].fillna(df_movie_cleaned_v1['Titre Original'])

C:\Users\koke7\AppData\Local\Temp\ipykernel_3628\4254248086.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movie_cleaned_v1["Durée (minutes)"] = pd.to_numeric(df_movie_cleaned_v1["Durée (minutes)"], errors="coerce") # Converti en numérique


In [21]:

#! === CLEANUP V2 ===


#* Supprimer les films avec une note inférieur à 5
df_movie_cleaned_v2 = df_movie_cleaned_v1[df_movie_cleaned_v1['Note'] > 5]

#* Supprimer les films avec des votes supérieur à 50000
df_movie_cleaned_v2 = df_movie_cleaned_v2[df_movie_cleaned_v2['Votes'] > 5000]

#* Supprimer les documentaires
df_movie_cleaned_v2 = df_movie_cleaned_v2[df_movie_cleaned_v2['Genre Principal'] != 'Documentary']

#* Supprimer les films avec une note inférieur à 8 et sortis avant 2000
df_movie_cleaned_v2 = df_movie_cleaned_v2[~((df_movie_cleaned_v2['Note'] < 8) & (df_movie_cleaned_v2['Année'] < 2014))]

## **Exportation**

In [23]:

#! === EXPORT ===

df_movie_cleaned_v2.to_csv("../data/processed/df_movie_cleaned.csv", index=False)

## **Rendu :**

In [22]:
df_movie_cleaned_v2

,ID,Réalisateur(s),Titre Original,Titre Français,Métrage,Durée (minutes),Année,Décennie,Genres,Genre Principal,Note,Votes,Popularité,Acteurs
105,tt0065670,Leonid Gaidai,Twelve Chairs,Les douze heures,Moyen,153.0,1971,70°s,"Adventure, Comedy, Crime",Adventure,8.2,7508,Connu,"Archil Gomiashvili, Sergey Filippov, Mikhail P..."
143,tt0065889,Elio Petri,Investigation of a Citizen Above Suspicion,Enquête sur un citoyen au-dessus de tout soupçon,Moyen,115.0,1970,70°s,"Crime, Drama",Crime,8.0,13890,Connu,"Gian Maria Volontè, Gian Maria Volontè, Gianni..."
176,tt0066078,Sergiu Nicolaescu,Michael the Brave,La dernière croisade,Long,203.0,1971,70°s,"Action, Biography, Drama",Action,8.5,6609,Connu,"Amza Pellea, Amza Pellea, Ion Besoiu, György K..."
289,tt0066763,Hrishikesh Mukherjee,Anand,Anand,Moyen,122.0,1971,70°s,"Drama, Musical",Drama,8.1,35953,Connu,"Rajesh Khanna, Amitabh Bachchan, Amitabh Bachc..."
333,tt0066921,Stanley Kubrick,A Clockwork Orange,Orange mécanique,Moyen,136.0,1971,70°s,"Crime, Sci-Fi",Crime,8.2,898263,Très populaire,"Malcolm McDowell, Patrick Magee, Michael Bates..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41601,tt9894470,Joe Begos,VFW,VFW,Court,92.0,2019,2010,"Action, Crime, Horror",Action,6.1,9837,Connu,"Stephen Lang, William Sadler, Fred Williamson,..."
41603,tt9898858,Michael Dowse,Coffee & Kareem,Coffee & Kareem,Court,88.0,2020,2020,"Action, Comedy, Crime",Action,5.2,14776,Connu,"Ed Helms, Terrence Little Gardenhigh, RonReaco..."
41604,tt9900782,Lokesh Kanagaraj,Kaithi,Kaithi,Moyen,145.0,2019,2010,"Action, Crime, Drama",Action,8.4,44902,Connu,"Karthi, Narain, Arjun Das, George Maryan, Hari..."
41608,tt9907782,Sean Ellis,The Cursed,Eight for Silver,Moyen,111.0,2021,2020,"Fantasy, Horror, Mystery",Fantasy,6.2,19027,Connu,"Boyd Holbrook, Alistair Petrie, Nigel Betts, S..."
